# Exporting search results
This notebook demonstrates how to export the results of a search using the API. Examples here will be shown using Python with the use of the `requests` library, though the same work could be done using the unofficial Python [ADS library](https://ads.readthedocs.io/en/latest/) or `curl` commands on the command line.

## Using the API
In all examples below, `token` should be replaced with [your own API token](https://ui.adsabs.harvard.edu/user/settings/token). If you haven't worked with our API before, it's recommended that you read the [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md) before beginning. We also assume some familiarity with our other API Jupyter notebooks, especially the ones on searching and exporting.

In [1]:
# import the requests package and set your token in a variable for later use
import requests
token="nlfpePC8LV4EQ5c9BXtk0VbnXKifgjVBnWOpa0az"
#token="your-token-here"

## Introduction
In this library, we demonstrate how to run a search, extract the bibcodes of the search results, and export those in the format of your choice.

## Searching with the API
Let's start with an example search:

`references(bibcode:1998ApJ...509..678G)`

This search finds the references from a well-cited paper by a recent Nobel Prize winner. To convert that to a search, we first take the base URL for searching in the API:

`https://api.adsabs.harvard.edu/v1/search/query?`

We'll also need to pick which fields we want to return; since we'll just be exporting all of the search results, let's only return the bibcode for each search result.

We then need to URL-encode our search terms before we add them to the URL. Python offers an easy way to do this.

In [2]:
from urllib.parse import urlencode
query = 'references(bibcode:1998ApJ...509..678G)'
# the "q" key below designates that this is the search query, while the "fl" refers to the fields to return
encoded_query = urlencode({'q': query, 'fl': 'bibcode'})
encoded_query

'q=references%28bibcode%3A1998ApJ...509..678G%29&fl=bibcode'

We'll append the encoded query terms to the base URL to create the URL we'll send our API request to.

Two other things you'll need to know:
* this API endpoint uses the GET HTTP method, so you'll use the `get` method on the request. (Note that you can find all API endpoints and their HTTP methods in our [full API documentation](https://ui.adsabs.harvard.edu/help/api/api-docs.html).)
* virtually all API requests require you to pass your API token in the header, as shown below

Putting all this together, we can now run our search through the API:

In [3]:
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})
# format the response in a nicely readable format
results.json()

{'responseHeader': {'status': 0,
  'QTime': 39,
  'params': {'q': 'references(bibcode:1998ApJ...509..678G)',
   'fl': 'bibcode',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dc03f7-27cbe4091fea494b56ea9a99',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 27,
  'start': 0,
  'docs': [{'bibcode': '1987gady.book.....B'},
   {'bibcode': '1995Natur.373..127M'},
   {'bibcode': '1993ARA&A..31..345R'},
   {'bibcode': '1996Natur.383..415E'},
   {'bibcode': '1997MNRAS.284..576E'},
   {'bibcode': '1994ApJ...435L..35H'},
   {'bibcode': '1997MNRAS.291..219G'},
   {'bibcode': '1971MNRAS.152..461L'},
   {'bibcode': '1994ApJ...435L..27F'},
   {'bibcode': '1996ApJ...472..153G'}]}}

Our query returned 10 results, which is the default number of results to return. However, there are a total of 27 results that match the search (see the `numFound` parameter in the response). We have a couple options to return all of the search results. We could increase the number of results returned to something equal to or larger than the total number of search results using the `rows` parameter (up to a maximum of 2000 rows; see the [full API documentation](https://ui.adsabs.harvard.edu/help/api/api-docs.html) or the search Jupyter notebook for more details). Alternatively, we can page through the list of results. 

Since the latter option is slightly more complex, we'll demonstrate that option here. While we're at it, we'll also extract the bibcodes from the dictionary structure they were returned in into a more manageable and useful list.

In [4]:
rows = 10 # fetch 10 records at a time
start = 0  # start with the first result
bibcodes = [] # we'll store the bibcodes of all of our results here

# this is the pagination - the while loop will automatically stop once we've fetched all docs
docs = True
while docs:
    # note that this URL is the same as above, except we've added parameters for start and rows
    results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{encoded_query}" \
                           "&rows={rows}" \
                           "&start={start}".format(encoded_query=encoded_query,rows=rows,start=start), \
                           headers={'Authorization': 'Bearer ' + token})
    try:
        docs = results.json()['response']['docs']
    except KeyError:
        print('No docs found')
        break
    # pull the bibcodes out of the results into a list
    tmp = [d['bibcode'] for d in docs]
    bibcodes = bibcodes + tmp
    start += rows # increment the start value to move to the next page of results

print(len(bibcodes))

27


Ok, we've now completed the search and extracted all of the search results into a list. Let's move on to exporting them.

## Exporting the search results
There are many options for export formats; see the [full API documentation](https://ui.adsabs.harvard.edu/help/api/api-docs.html) for a list of options. 

### Export to BibTeX

One of the most commonly used export formats in ADS is BibTeX, so let's start with that one. The base URL for a BibTeX export is:

`https://api.adsabs.harvard.edu/v1/export/bibtex`

Other export formats follow the same pattern, with the name of the export format replacing `bibtex` in the above URL.

This URL can be used with either the GET or the POST HTTP method, depending on your needs. 

#### Export a single bibcode

If you want to export a single record, you'll use the GET method, putting the bibcode at the end of the URL:

In [5]:
# let's export the first record from our list of bibcodes
bibcode = bibcodes[0]

results = requests.get("https://api.adsabs.harvard.edu/v1/export/bibtex/{bibcode}".format(bibcode=bibcode),
                      headers={'Authorization': 'Bearer ' + token})


results.text

'@BOOK{1987gady.book.....B,\n       author = {{Binney}, James and {Tremaine}, Scott},\n        title = "{Galactic dynamics}",\n         year = 1987,\n       adsurl = {https://ui.adsabs.harvard.edu/abs/1987gady.book.....B},\n      adsnote = {Provided by the SAO/NASA Astrophysics Data System}\n}\n\n'

#### Export a list of bibcodes

To export the entire list of bibcodes, you'll use the POST method, with the list of bibcodes passed in the payload:

In [6]:
# create a dictionary with the payload values
payload = {'bibcode': bibcodes}

# the json library offers an easy way to convert between JSON or dictionaries and their serialized strings
import json
serialized_payload = json.dumps(payload)

results = requests.post("https://api.adsabs.harvard.edu/v1/export/bibtex", 
                        headers={'Authorization': 'Bearer ' + token},
                        data=serialized_payload)

results.json()

{'msg': 'Retrieved 27 abstracts, starting with number 1.',
 'export': '@ARTICLE{1998ApJ...494L.181M,\n       author = {{Maoz}, Eyal},\n        title = "{Dynamical Constraints on Alternatives to Supermassive Black Holes in Galactic Nuclei}",\n      journal = {\\apjl},\n     keywords = {BLACK HOLE PHYSICS, GALAXIES: KINEMATICS AND DYNAMICS, GALAXIES: NUCLEI, Black Hole Physics, Galaxies: Kinematics and Dynamics, Galaxies: Nuclei, Astrophysics},\n         year = 1998,\n        month = feb,\n       volume = {494},\n       number = {2},\n        pages = {L181-L184},\n          doi = {10.1086/311194},\narchivePrefix = {arXiv},\n       eprint = {astro-ph/9710309},\n primaryClass = {astro-ph},\n       adsurl = {https://ui.adsabs.harvard.edu/abs/1998ApJ...494L.181M},\n      adsnote = {Provided by the SAO/NASA Astrophysics Data System}\n}\n\n@ARTICLE{1997ApJ...490L..77S,\n       author = {{Serabyn}, E. and {Carlstrom}, J. and {Lay}, O. and {Lis}, D.~C. and {Hunter}, T.~R. and {Lacy}, J.~H. and {

The results of the POST query come back as a dictionary; to get just the text, extract it from the dictionary:

In [7]:
results.json()['export']

'@ARTICLE{1998ApJ...494L.181M,\n       author = {{Maoz}, Eyal},\n        title = "{Dynamical Constraints on Alternatives to Supermassive Black Holes in Galactic Nuclei}",\n      journal = {\\apjl},\n     keywords = {BLACK HOLE PHYSICS, GALAXIES: KINEMATICS AND DYNAMICS, GALAXIES: NUCLEI, Black Hole Physics, Galaxies: Kinematics and Dynamics, Galaxies: Nuclei, Astrophysics},\n         year = 1998,\n        month = feb,\n       volume = {494},\n       number = {2},\n        pages = {L181-L184},\n          doi = {10.1086/311194},\narchivePrefix = {arXiv},\n       eprint = {astro-ph/9710309},\n primaryClass = {astro-ph},\n       adsurl = {https://ui.adsabs.harvard.edu/abs/1998ApJ...494L.181M},\n      adsnote = {Provided by the SAO/NASA Astrophysics Data System}\n}\n\n@ARTICLE{1997ApJ...490L..77S,\n       author = {{Serabyn}, E. and {Carlstrom}, J. and {Lay}, O. and {Lis}, D.~C. and {Hunter}, T.~R. and {Lacy}, J.~H. and {Hills}, R.~E.},\n        title = "{High-Frequency Measurements of the 

### Export using a custom format
You can also export your results using a custom format. We offer many options for custom export formatting; see all of them [here](https://ui.adsabs.harvard.edu/help/actions/export#the-ads-custom-format).

We'll use one of the custom formats from the help pages, which formats a record into a readable bibliography-type format:

`%l (%Y), %j, %V, %p.\n`

The custom format base URL is:

`https://api.adsabs.harvard.edu/v1/export/custom`

and it always uses the POST HTTP method. The payload requires both the list of bibcodes to export as well as the custom format string. You can choose to include a sort order as well:

In [8]:
custom_format="%l (%Y), %j, %V, %p.\n"

payload = {'bibcode': bibcodes,
           'format': custom_format,
           'sort': 'first_author asc'}

serialized_payload = json.dumps(payload)

results = requests.post("https://api.adsabs.harvard.edu/v1/export/custom", 
                        headers={'Authorization': 'Bearer ' + token},
                        data=serialized_payload)

results.json()

{'msg': 'Retrieved 27 abstracts, starting with number 1.',
 'export': 'Backer, D. C. (1996), Unsolved Problems of the Milky Way, 169, 193.\n\nBahcall, J. N., & Tremaine, S. (1981), \\apj, 244, 805.\n\nBeckert, T., Duschl, W. J., Mezger, P. G., & Zylka, R. (1996), \\aap, 307, 450.\n\nBinney, J., & Tremaine, S. (1987), Princeton,.\n\nChristou, J. C. (1991), \\pasp, 103, 1040.\n\nEckart, A., & Genzel, R. (1996), \\nat, 383, 415.\n\nEckart, A., & Genzel, R. (1997), \\mnras, 284, 576.\n\nFord, H. C., Harms, R. J., Tsvetanov, Z. I., Hartig, G. F., Dressel, L. L., Kriss, G. A., Bohlin, R. C., Davidsen, A. F., Margon, B., & Kochhar, A. K. (1994), \\apjl, 435, L27.\n\nGenzel, R., Eckart, A., Ott, T., & Eisenhauer, F. (1997), \\mnras, 291, 219.\n\nGenzel, R., Thatte, N., Krabbe, A., Kroker, H., & Tacconi-Garman, L. E. (1996), \\apj, 472, 153.\n\nGreenhill, L. J., Jiang, D. R., Moran, J. M., Reid, M. J., Lo, K. Y., & Claussen, M. J. (1995), \\apj, 440, 619.\n\nGuesten, R., Genzel, R., Wright, M. 

In [9]:
print(results.json()['export'])

Backer, D. C. (1996), Unsolved Problems of the Milky Way, 169, 193.

Bahcall, J. N., & Tremaine, S. (1981), \apj, 244, 805.

Beckert, T., Duschl, W. J., Mezger, P. G., & Zylka, R. (1996), \aap, 307, 450.

Binney, J., & Tremaine, S. (1987), Princeton,.

Christou, J. C. (1991), \pasp, 103, 1040.

Eckart, A., & Genzel, R. (1996), \nat, 383, 415.

Eckart, A., & Genzel, R. (1997), \mnras, 284, 576.

Ford, H. C., Harms, R. J., Tsvetanov, Z. I., Hartig, G. F., Dressel, L. L., Kriss, G. A., Bohlin, R. C., Davidsen, A. F., Margon, B., & Kochhar, A. K. (1994), \apjl, 435, L27.

Genzel, R., Eckart, A., Ott, T., & Eisenhauer, F. (1997), \mnras, 291, 219.

Genzel, R., Thatte, N., Krabbe, A., Kroker, H., & Tacconi-Garman, L. E. (1996), \apj, 472, 153.

Greenhill, L. J., Jiang, D. R., Moran, J. M., Reid, M. J., Lo, K. Y., & Claussen, M. J. (1995), \apj, 440, 619.

Guesten, R., Genzel, R., Wright, M. C. H., Jaffe, D. T., Stutzki, J., & Harris, A. I. (1987), \apj, 318, 124.

Haller, J. W., Rieke, M. J.

That should be everything you need to know to search for papers and export them to the format of your choice. If you need more help, check out the rest of our [API documentation](https://ui.adsabs.harvard.edu/help/api/) or [email us](mailto:adshelp@cfa.harvard.edu).